In [1]:
from setfit import SetFitModel
from setfit.data import get_templated_dataset
from setfit.exporters.onnx import export_onnx
from setfit.trainer import SetFitTrainer

There was a problem when trying to write in your cache folder (/home/nghiatl/.cache/huggingface/hub). You should set the environment variable TRANSFORMERS_CACHE to a writable directory.


In [2]:
from transformers import AutoTokenizer

In [3]:
model_path_2 = "sentence-transformers/paraphrase-albert-small-v2"

In [4]:
model_path = "./setfitckpt_multilingual_test_en_v1_af"

In [8]:
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-multilingual-mpnet-base-v2",use_differentiable_head=True, head_params={"out_features": 3})

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [10]:
model_2 = SetFitModel.from_pretrained(model_path_2, use_differentiable_head=True, head_params={"out_features": 3})

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [9]:
model.model_body._modules["1"]

Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})

In [11]:
model_2.model_body._modules["1"]

Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})

In [12]:
model_2.model_head

SetFitHead({'in_features': 768, 'out_features': 3, 'temperature': 1.0, 'bias': True, 'device': 'cuda'})

In [13]:
model.model_head

SetFitHead({'in_features': 768, 'out_features': 3, 'temperature': 1.0, 'bias': True, 'device': 'cuda'})

In [14]:
next(model.model_body.parameters())

Parameter containing:
tensor([[ 0.0662,  0.0555,  0.0743,  ...,  0.0534,  0.0464,  0.0255],
        [-0.0026,  0.0016, -0.0028,  ...,  0.0028,  0.0014, -0.0028],
        [ 0.0888,  0.0866,  0.0601,  ...,  0.0713, -0.0012,  0.0890],
        ...,
        [ 0.1257, -0.1544,  0.0596,  ...,  0.0792,  0.0191,  0.0341],
        [ 0.0070, -0.0440,  0.0062,  ..., -0.0052,  0.0124, -0.0409],
        [ 0.0358,  0.0211,  0.0197,  ...,  0.0437, -0.0023,  0.0433]],
       device='cuda:0', requires_grad=True)

In [15]:
import onnxruntime

In [18]:
import os

# output_path = "model.onnx"
# # try:
# export_onnx(model_2.model_body, model_2.model_head, opset=12, output_path=output_path)

# # Check that the model was saved.
# assert output_path in os.listdir(), "Model not saved to output_path"

# Run inference using the original model.
print('lets predict')
input_text = ["i loved the spiderman movie!", "pineapple on pizza is the worst 🤮"]

#     pytorch_preds = model(input_text)

# Run inference using the exported onnx model.
tokenizer = AutoTokenizer.from_pretrained(model_path_2)
inputs = tokenizer(
    input_text,
    padding=True,
    truncation=True,
    return_attention_mask=True,
    return_token_type_ids=True,
    return_tensors="np",
)
# Map inputs to int64 from int32
inputs = {key: value.astype("int64") for key, value in inputs.items()}

session = onnxruntime.InferenceSession(output_path)

onnx_preds = session.run(None, dict(inputs))[0]
    
    # Compare the results and ensure that we get the same predictions.
    # assert np.array_equal(onnx_preds, pytorch_preds)

# finally:
#     # Cleanup the model.
#     os.remove(output_path)

lets predict


In [19]:
onnx_preds

array([[-0.33376396, -0.8639664 ,  0.43391865],
       [ 0.68041843, -0.8529784 , -0.54821074]], dtype=float32)